In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
import warnings

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [44]:
warnings.filterwarnings("ignore")

In [45]:
# решаем проблему с кодировкой
data = pd.read_csv('complete laptop data0.csv',encoding='unicode_escape')
data.info()
# cols

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 98 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         984 non-null    int64  
 1   link                               984 non-null    object 
 2   name                               984 non-null    object 
 3   user rating                        690 non-null    float64
 4   Price                              984 non-null    object 
 5   Sales Package                      984 non-null    object 
 6   Model Number                       984 non-null    object 
 7   Part Number                        984 non-null    object 
 8   Model Name                         709 non-null    object 
 9   Series                             787 non-null    object 
 10  Color                              984 non-null    object 
 11  Type                               984 non-null    object 

In [46]:
cols = data.columns.tolist()
cols.remove('Processor Brand')
cols.remove('Processor Name')
cols.remove('Processor Generation')
cols.remove('Processor Variant')
cols.remove("Number of Cores")

data=data.drop(cols,axis=1)
data.head()

,Processor Brand,Processor Name,Processor Generation,Processor Variant,Number of Cores
0,Intel,Core i9,12th Gen,12900H,14.0
1,Intel,Core i9,12th Gen,12900H,14.0
2,AMD,Ryzen 7 Octa Core,NaN,5800H,8.0
3,AMD,Ryzen 7 Octa Core,5th Gen,R7-5800H,8.0
4,Intel,Core i5,11th Gen,NaN,NaN


In [47]:
data["Processor Name"].isna().sum()

0

In [48]:
data["Processor Variant"].isna().sum()

123

In [54]:
data[data["Processor Generation"].isna() & data["Number of Cores"].isna()]

,Processor Brand,Processor Name,Processor Generation,Processor Variant,Number of Cores
6,Intel,Celeron Dual Core,NaN,NaN,NaN
23,Intel,Celeron Dual Core,NaN,N4020,NaN
78,Intel,Celeron Dual Core,NaN,NaN,NaN
95,Intel,Celeron Dual Core,NaN,N4020,NaN
99,AMD,Ryzen 5 Quad Core,NaN,3500U,NaN
110,AMD,Ryzen 7 Quad Core,NaN,3700U,NaN
130,AMD,Ryzen 5 Hexa Core,NaN,5600H,NaN
148,AMD,Ryzen 3 Dual Core,NaN,NaN,NaN
161,Qualcomm,Snapdragon 7c Gen 2,NaN,NaN,NaN
168,AMD,Ryzen 5 Quad Core,NaN,NaN,NaN


In [50]:
data[data["Processor Generation"].isna() & data["Processor Name"].isna()]

,Processor Brand,Processor Name,Processor Generation,Processor Variant,Number of Cores


In [55]:
data["Processor Name"].nunique()

30

In [52]:
data[data["Processor Generation"].isna()]["Processor Name"].unique()

array(['Ryzen 7 Octa Core', 'Celeron Dual Core', 'Pentium Silver',
       'Athlon Dual Core', 'Ryzen 5 Hexa Core', 'Ryzen 5 Quad Core',
       'Pentium Quad Core', 'Celeron Quad Core', 'Ryzen 7 Quad Core',
       'Snapdragon 7c Gen 2', 'Ryzen 5 Dual Core', 'Ryzen 3 Dual Core',
       'Ryzen 9 Octa Core', 'Dual Core', 'Ryzen 3 Quad Core',
       'Ryzen 7 Hexa Core', 'Ryzen 5 Octa Core', 'M1 Pro',
       'APU Dual Core A6', 'MediaTek Kompanio 500', 'M1',
       'APU Dual Core A9'], dtype=object)

In [53]:
data[data["Number of Cores"].isna()]["Processor Name"].unique()

array(['Core i5', 'Celeron Dual Core', 'Ryzen 7 Octa Core',
       'Ryzen 7 Dual Core', 'Ryzen 3 Quad Core', 'Core i7',
       'Ryzen 7 Hexa Core', 'Ryzen 9 Octa Core', 'Ryzen 7 Quad Core',
       'Ryzen 5 Quad Core', 'Ryzen 5 Hexa Core', 'Core i3',
       'Ryzen 3 Dual Core', 'Pentium Quad Core', 'Snapdragon 7c Gen 2',
       'Dual Core', 'Ryzen 5 Octa Core', 'Core i9', 'Athlon Dual Core',
       'Ryzen 3 Hexa Core', 'Octa Core i7'], dtype=object)